In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import onnxmltools

In [2]:
model = Sequential([
    Dense(32, activation='relu', input_dim=100,  name='main_input'),
    Dense(1, activation='sigmoid', name="dense_1"),
])

In [3]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [4]:
data = np.random.random((1000, 100))
labels = np.random.randint(2, size=(1000, 1))

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, labels, epochs=10, batch_size=32)

Epoch 1/10
32/32 [==============================] - 0s 2ms/step - loss: 0.7168 - accuracy: 0.4960
Epoch 2/10
32/32 [==============================] - 0s 1ms/step - loss: 0.6988 - accuracy: 0.5200
Epoch 3/10
32/32 [==============================] - 0s 2ms/step - loss: 0.6935 - accuracy: 0.5250
Epoch 4/10
32/32 [==============================] - 0s 2ms/step - loss: 0.6860 - accuracy: 0.5530
Epoch 5/10
32/32 [==============================] - 0s 1ms/step - loss: 0.6798 - accuracy: 0.5660
Epoch 6/10
32/32 [==============================] - 0s 1ms/step - loss: 0.6737 - accuracy: 0.5780
Epoch 7/10
32/32 [==============================] - 0s 2ms/step - loss: 0.6714 - accuracy: 0.5710
Epoch 8/10
32/32 [==============================] - 0s 1ms/step - loss: 0.6645 - accuracy: 0.5900
Epoch 9/10
32/32 [==============================] - 0s 1ms/step - loss: 0.6639 - accuracy: 0.6070
Epoch 10/10
32/32 [==============================] - 0s 1ms/step - loss: 0.6574 - accuracy: 0.6180


In [5]:
#Save in tf's format, for benchmarking
model.save('my_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_model/assets


In [6]:
#Saving it in ONNX format makes it easier to 
#pass to Rust
onnx_model = onnxmltools.convert_keras(model)
onnxmltools.utils.save_model(onnx_model, 'example.onnx')

tf executing eager_mode: True
tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 6
The maximum opset needed by this model is only 9.
